In [13]:
from fyers_apiv3 import fyersModel
import os
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
import numpy as np
from datetime import timedelta
import math
from fyers_apiv3.FyersWebsocket import data_ws
from sklearn.linear_model import LinearRegression
with open("abcd.txt",'r') as r:
    access_token=r.read()
client_id = "DZO41L3M36-100" 
fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path=os.getcwd())
fyers.get_profile()
def calculate_slope_angle(x, y):
    model = LinearRegression().fit(x, y)
    slope = model.coef_[0]
    angle_deg = math.degrees(math.atan(slope))
    return slope, angle_deg


In [14]:
interval=5
symbol="NSE:NIFTYBANK-INDEX"
data = {
    "symbol":"NSE:NIFTYBANK-INDEX",
    "resolution":"5",
    "date_format":"1", 
    "range_from":"2024-07-06",
    "range_to":"2024-08-06",
    "cont_flag":"1",
    "interval":"5",
}
response = fyers.history(data=data)

# Check response structure
if 'candles' in response:
    df = pd.DataFrame(response['candles'])
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.date = (df.date.dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata'))
    df['date'] = df['date'].dt.tz_localize(None)
    #df.set_index('date', inplace=True)
    n=len(df)
else:
    print("Error: 'candles' key not found in response")
    print("Response:", response)
    df = pd.DataFrame()  # Create an empty DataFrame to avoid further errors
df


In [15]:
import plotly.graph_objects as go

if not df.empty:
    # Filter out weekends
    
    fig = go.Figure(data=[go.Candlestick(x=df.date,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'])])


    """
    # Plot candlesticks for trading hours only
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['close'],
                                 name='Market Hours'))
    
    """
    # Define time bounds for range breaks
    if symbol[:3] == 'MCX' :
        bounds = [24,9]
    elif interval == '1440':
        bounds = [12,18]
    elif interval == '5':
        bounds = [15.5,9.25]
    elif interval == '15':
        bounds = [15.5,9.25]
    elif interval == '30':
        bounds = [15.75,9.25]
    else :
        bounds = [16,9.25]
    alldays =set(df.date[0]+timedelta(x) for x in range((df.date[len(df.date)-1]- df.date[0]).days))
    missing=sorted(set(alldays)-set(df.date))
    fig.update_xaxes(
        rangeslider_visible=False,
        rangebreaks=[
            dict(bounds=bounds, pattern="hour"),
            dict(values=missing)
        ]
    )

    fig
else:
    print("DataFrame is empty. Cannot proceed with analysis.")

In [16]:
def detect_consecutive_ibars(df, max_inside_bars=12, height_ratio_threshold=0.6, similar_height_threshold=0.4, max_similar_height=2, body_to_wick_ratio=0.7):
    consecutive_ibars = [False] * len(df)
    i = 0
    while i < len(df) - 1:
        previous_candle = df.iloc[i]
        main_height = previous_candle['high'] - previous_candle['low']
        main_body = abs(previous_candle['open'] - previous_candle['close'])
        main_wick = main_height - main_body

        # Check if the body is significantly larger than the wicks
        if main_wick == 0 or main_body / main_wick < body_to_wick_ratio:
            i += 1
            continue

        # Start looking for inside bars after the current candle
        j = i + 1
        count = 0
        similar_height_count = 0
        tall_inside_count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            inside_height = df.iloc[j]['high'] - df.iloc[j]['low']
            if inside_height >= main_height * height_ratio_threshold:
                similar_height_count += 1
                if similar_height_count > max_similar_height:
                    break

            if inside_height >= main_height * similar_height_threshold:
                tall_inside_count += 1

            count += 1
            j += 1
        
        # Check if the count of inside bars is within the allowed limit
        if count > 3 and count <= max_inside_bars and similar_height_count <= max_similar_height:
            if tall_inside_count / count < 0.5:  # Check if less than 50% of inside bars are tall
                for k in range(i+1, j):
                    consecutive_ibars[k] = True
                # Mark the larger candlestick as well
                consecutive_ibars[i] = True
        # Move index to the end of the current sequence
        i = j if count == 0 else i + 1
                
    return consecutive_ibars

df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)

# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty
    else:
        i += 1

# Show the figure
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([52386.2 , 52606.5 , 52599.45, ..., 49728.05, 49758.95, 49770.2 ]),
              'high': array([52534.1 , 52642.4 , 52656.55, ..., 49755.35, 49771.  , 49797.55]),
              'low': array([52321.2 , 52370.1 , 52555.65, ..., 49708.05, 49717.2 , 49755.55]),
              'open': array([52533.1 , 52384.35, 52606.45, ..., 49738.45, 49732.5 , 49755.55]),
              'type': 'candlestick',
              'x': array([datetime.datetime(2024, 7, 8, 9, 15),
                          datetime.datetime(2024, 7, 8, 9, 20),
                          datetime.datetime(2024, 7, 8, 9, 25), ...,
                          datetime.datetime(2024, 8, 6, 15, 15),
                          datetime.datetime(2024, 8, 6, 15, 20),
                          datetime.datetime(2024, 8, 6, 15, 25)], dtype=object)}],
    'layout': {'shapes': [{'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 09:55:00'),
                           'x1': Timestamp('2024-07-08 10:25:00'),
                           'y0': 52424.9,
                           'y1': 52628.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 10:25:00'),
                           'x1': Timestamp('2024-07-08 11:05:00'),
                           'y0': 52461.35,
                           'y1': 52676.55},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-09 15:25:00'),
                           'x1': Timestamp('2024-07-10 10:20:00'),
                           'y0': 52175.2,
                           'y1': 52528.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-11 13:55:00'),
                           'x1': Timestamp('2024-07-11 14:50:00'),
                           'y0': 52074.8,
                           'y1': 52244.7},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-15 13:40:00'),
                           'x1': Timestamp('2024-07-15 14:10:00'),
                           'y0': 52543.3,
                           'y1': 52662.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-23 09:35:00'),
                           'x1': Timestamp('2024-07-23 10:30:00'),
                           'y0': 52088.55,
                           'y1': 52321.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-24 10:20:00'),
                           'x1': Timestamp('2024-07-24 10:55:00'),
                           'y0': 51361.35,
                           'y1': 51720.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-29 09:45:00'),
                           'x1': Timestamp('2024-07-29 10:45:00'),
                           'y0': 51751.15,
                           'y1': 51954.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
      

In [17]:
def detect_consecutive_ibars(df, max_inside_bars=12, height_ratio_threshold=0.75, similar_height_threshold=0.6, max_similar_height=3, body_to_wick_ratio=0.7, tall_inside_threshold=0.5, tall_inside_percent=0.3):
    consecutive_ibars = [False] * len(df)
    i = 0
    while i < len(df) - 1:
        previous_candle = df.iloc[i]
        main_height = previous_candle['high'] - previous_candle['low']
        main_body = abs(previous_candle['open'] - previous_candle['close'])
        main_wick = main_height - main_body

        # Check if the body is significantly larger than the wicks
        if main_wick == 0 or main_body / main_wick < body_to_wick_ratio:
            i += 1
            continue

        # Start looking for inside bars after the current candle
        j = i + 1
        count = 0
        similar_height_count = 0
        tall_inside_count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            inside_height = df.iloc[j]['high'] - df.iloc[j]['low']
            if inside_height >= main_height * height_ratio_threshold:
                similar_height_count += 1
                if similar_height_count > max_similar_height:
                    break

            if inside_height >= main_height * similar_height_threshold:
                tall_inside_count += 1
            
            if inside_height >= main_height * tall_inside_threshold:
                tall_inside_count += 1

            count += 1
            j += 1
        
        # Check if the count of inside bars is within the allowed limit
        if count > 3 and count <= max_inside_bars and similar_height_count <= max_similar_height:
            if tall_inside_count / count < 0.5 and tall_inside_count / count < tall_inside_percent:  # Check if less than 50% of inside bars are tall and less than 30% of inside bars are taller than 50% of main candlestick
                for k in range(i+1, j):
                    consecutive_ibars[k] = True
                # Mark the larger candlestick as well
                consecutive_ibars[i] = True
        # Move index to the end of the current sequence
        i = j if count == 0 else i + 1
                
    return consecutive_ibars

df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)

# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty
    else:
        i += 1

# Show the figure
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([52386.2 , 52606.5 , 52599.45, ..., 49728.05, 49758.95, 49770.2 ]),
              'high': array([52534.1 , 52642.4 , 52656.55, ..., 49755.35, 49771.  , 49797.55]),
              'low': array([52321.2 , 52370.1 , 52555.65, ..., 49708.05, 49717.2 , 49755.55]),
              'open': array([52533.1 , 52384.35, 52606.45, ..., 49738.45, 49732.5 , 49755.55]),
              'type': 'candlestick',
              'x': array([datetime.datetime(2024, 7, 8, 9, 15),
                          datetime.datetime(2024, 7, 8, 9, 20),
                          datetime.datetime(2024, 7, 8, 9, 25), ...,
                          datetime.datetime(2024, 8, 6, 15, 15),
                          datetime.datetime(2024, 8, 6, 15, 20),
                          datetime.datetime(2024, 8, 6, 15, 25)], dtype=object)}],
    'layout': {'shapes': [{'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 09:55:00'),
                           'x1': Timestamp('2024-07-08 10:25:00'),
                           'y0': 52424.9,
                           'y1': 52628.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 10:25:00'),
                           'x1': Timestamp('2024-07-08 11:05:00'),
                           'y0': 52461.35,
                           'y1': 52676.55},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-09 15:25:00'),
                           'x1': Timestamp('2024-07-10 10:20:00'),
                           'y0': 52175.2,
                           'y1': 52528.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-11 13:55:00'),
                           'x1': Timestamp('2024-07-11 14:50:00'),
                           'y0': 52074.8,
                           'y1': 52244.7},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-15 13:40:00'),
                           'x1': Timestamp('2024-07-15 14:10:00'),
                           'y0': 52543.3,
                           'y1': 52662.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-23 09:35:00'),
                           'x1': Timestamp('2024-07-23 10:30:00'),
                           'y0': 52088.55,
                           'y1': 52321.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-24 10:20:00'),
                           'x1': Timestamp('2024-07-24 10:55:00'),
                           'y0': 51361.35,
                           'y1': 51720.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-29 09:45:00'),
                           'x1': Timestamp('2024-07-29 10:45:00'),
                           'y0': 51751.15,
                           'y1': 51954.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
      

In [18]:

def detect_consecutive_ibars(df, max_inside_bars=12, height_ratio_threshold=0.75, max_similar_height=3, body_to_wick_ratio=0.5):
    consecutive_ibars = [False] * len(df)
    i = 0
    while i < len(df) - 1:
        previous_candle = df.iloc[i]
        main_height = previous_candle['high'] - previous_candle['low']
        main_body = abs(previous_candle['open'] - previous_candle['close'])
        main_wick = main_height - main_body

        # Check if the body is significantly larger than the wicks
        if main_wick == 0 or main_body / main_wick < body_to_wick_ratio:
            i += 1
            continue

        # Start looking for inside bars after the current candle
        j = i + 1
        count = 0
        similar_height_count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            inside_height = df.iloc[j]['high'] - df.iloc[j]['low']
            if inside_height >= main_height * height_ratio_threshold:
                similar_height_count += 1
                if similar_height_count > max_similar_height:
                    break

            count += 1
            j += 1
        
        # Check if the count of inside bars is within the allowed limit
        if count > 3 and count <= max_inside_bars and similar_height_count <= max_similar_height:
            for k in range(i+1, j):
                consecutive_ibars[k] = True
            # Mark the larger candlestick as well
            consecutive_ibars[i] = True
        # Move index to the end of the current sequence
        i = j if count == 0 else i + 1
                
    return consecutive_ibars

df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)

# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty
    else:
        i += 1

# Show the figure
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([52386.2 , 52606.5 , 52599.45, ..., 49728.05, 49758.95, 49770.2 ]),
              'high': array([52534.1 , 52642.4 , 52656.55, ..., 49755.35, 49771.  , 49797.55]),
              'low': array([52321.2 , 52370.1 , 52555.65, ..., 49708.05, 49717.2 , 49755.55]),
              'open': array([52533.1 , 52384.35, 52606.45, ..., 49738.45, 49732.5 , 49755.55]),
              'type': 'candlestick',
              'x': array([datetime.datetime(2024, 7, 8, 9, 15),
                          datetime.datetime(2024, 7, 8, 9, 20),
                          datetime.datetime(2024, 7, 8, 9, 25), ...,
                          datetime.datetime(2024, 8, 6, 15, 15),
                          datetime.datetime(2024, 8, 6, 15, 20),
                          datetime.datetime(2024, 8, 6, 15, 25)], dtype=object)}],
    'layout': {'shapes': [{'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 09:55:00'),
                           'x1': Timestamp('2024-07-08 10:25:00'),
                           'y0': 52424.9,
                           'y1': 52628.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 10:25:00'),
                           'x1': Timestamp('2024-07-08 11:05:00'),
                           'y0': 52461.35,
                           'y1': 52676.55},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-09 15:25:00'),
                           'x1': Timestamp('2024-07-10 10:20:00'),
                           'y0': 52175.2,
                           'y1': 52528.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-11 13:55:00'),
                           'x1': Timestamp('2024-07-11 14:50:00'),
                           'y0': 52074.8,
                           'y1': 52244.7},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-15 13:40:00'),
                           'x1': Timestamp('2024-07-15 14:10:00'),
                           'y0': 52543.3,
                           'y1': 52662.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-23 09:35:00'),
                           'x1': Timestamp('2024-07-23 10:30:00'),
                           'y0': 52088.55,
                           'y1': 52321.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-24 10:20:00'),
                           'x1': Timestamp('2024-07-24 10:55:00'),
                           'y0': 51361.35,
                           'y1': 51720.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-29 09:45:00'),
                           'x1': Timestamp('2024-07-29 10:45:00'),
                           'y0': 51751.15,
                           'y1': 51954.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
      

In [19]:
def detect_consecutive_ibars(df, max_inside_bars=12, height_ratio_threshold=0.75, max_similar_height=3):
    consecutive_ibars = [False] * len(df)
    i = 0
    while i < len(df) - 1:
        previous_candle = df.iloc[i]
        main_height = previous_candle['high'] - previous_candle['low']

        # Start looking for inside bars after the current candle
        j = i + 1
        count = 0
        similar_height_count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            inside_height = df.iloc[j]['high'] - df.iloc[j]['low']
            if inside_height >= main_height * height_ratio_threshold:
                similar_height_count += 1
                if similar_height_count > max_similar_height:
                    break

            count += 1
            j += 1
        
        # Check if the count of inside bars is within the allowed limit
        if count > 3 and count <= max_inside_bars and similar_height_count <= max_similar_height:
            for k in range(i+1, j):
                consecutive_ibars[k] = True
            # Mark the larger candlestick as well
            consecutive_ibars[i] = True
        # Move index to the end of the current sequence
        i = j if count == 0 else i + 1
                
    return consecutive_ibars

df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)

# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty
    else:
        i += 1

# Show the figure
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([52386.2 , 52606.5 , 52599.45, ..., 49728.05, 49758.95, 49770.2 ]),
              'high': array([52534.1 , 52642.4 , 52656.55, ..., 49755.35, 49771.  , 49797.55]),
              'low': array([52321.2 , 52370.1 , 52555.65, ..., 49708.05, 49717.2 , 49755.55]),
              'open': array([52533.1 , 52384.35, 52606.45, ..., 49738.45, 49732.5 , 49755.55]),
              'type': 'candlestick',
              'x': array([datetime.datetime(2024, 7, 8, 9, 15),
                          datetime.datetime(2024, 7, 8, 9, 20),
                          datetime.datetime(2024, 7, 8, 9, 25), ...,
                          datetime.datetime(2024, 8, 6, 15, 15),
                          datetime.datetime(2024, 8, 6, 15, 20),
                          datetime.datetime(2024, 8, 6, 15, 25)], dtype=object)}],
    'layout': {'shapes': [{'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 09:55:00'),
                           'x1': Timestamp('2024-07-08 10:25:00'),
                           'y0': 52424.9,
                           'y1': 52628.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 10:25:00'),
                           'x1': Timestamp('2024-07-08 11:05:00'),
                           'y0': 52461.35,
                           'y1': 52676.55},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-09 15:25:00'),
                           'x1': Timestamp('2024-07-10 10:20:00'),
                           'y0': 52175.2,
                           'y1': 52528.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-11 13:55:00'),
                           'x1': Timestamp('2024-07-11 14:50:00'),
                           'y0': 52074.8,
                           'y1': 52244.7},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-15 13:40:00'),
                           'x1': Timestamp('2024-07-15 14:10:00'),
                           'y0': 52543.3,
                           'y1': 52662.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-23 09:35:00'),
                           'x1': Timestamp('2024-07-23 10:30:00'),
                           'y0': 52088.55,
                           'y1': 52321.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-24 10:20:00'),
                           'x1': Timestamp('2024-07-24 10:55:00'),
                           'y0': 51361.35,
                           'y1': 51720.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-29 09:45:00'),
                           'x1': Timestamp('2024-07-29 10:45:00'),
                           'y0': 51751.15,
                           'y1': 51954.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
      

In [20]:
def detect_consecutive_ibars(df, max_inside_bars=12, height_ratio_threshold=0.75, max_similar_height=3):
    consecutive_ibars = [False] * len(df)
    i = 0
    while i < len(df) - 1:
        previous_candle = df.iloc[i]
        main_height = previous_candle['high'] - previous_candle['low']

        # Start looking for inside bars after the current candle
        j = i + 1
        count = 0
        similar_height_count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            inside_height = df.iloc[j]['high'] - df.iloc[j]['low']
            if inside_height >= main_height * height_ratio_threshold:
                similar_height_count += 1
                if similar_height_count > max_similar_height:
                    break

            count += 1
            j += 1
        
        # Check if the count of inside bars is within the allowed limit
        if count > 3 and count <= max_inside_bars and similar_height_count <= max_similar_height:
            for k in range(i+1, j):
                consecutive_ibars[k] = True
            # Mark the larger candlestick as well
            consecutive_ibars[i] = True
        # Move index to the end of the current sequence
        i = j
                
    return consecutive_ibars

df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)

# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty
    else:
        i += 1

# Show the figure
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'close': array([52386.2 , 52606.5 , 52599.45, ..., 49728.05, 49758.95, 49770.2 ]),
              'high': array([52534.1 , 52642.4 , 52656.55, ..., 49755.35, 49771.  , 49797.55]),
              'low': array([52321.2 , 52370.1 , 52555.65, ..., 49708.05, 49717.2 , 49755.55]),
              'open': array([52533.1 , 52384.35, 52606.45, ..., 49738.45, 49732.5 , 49755.55]),
              'type': 'candlestick',
              'x': array([datetime.datetime(2024, 7, 8, 9, 15),
                          datetime.datetime(2024, 7, 8, 9, 20),
                          datetime.datetime(2024, 7, 8, 9, 25), ...,
                          datetime.datetime(2024, 8, 6, 15, 15),
                          datetime.datetime(2024, 8, 6, 15, 20),
                          datetime.datetime(2024, 8, 6, 15, 25)], dtype=object)}],
    'layout': {'shapes': [{'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 09:55:00'),
                           'x1': Timestamp('2024-07-08 10:25:00'),
                           'y0': 52424.9,
                           'y1': 52628.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-08 10:25:00'),
                           'x1': Timestamp('2024-07-08 11:05:00'),
                           'y0': 52461.35,
                           'y1': 52676.55},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-09 15:25:00'),
                           'x1': Timestamp('2024-07-10 10:20:00'),
                           'y0': 52175.2,
                           'y1': 52528.8},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-11 13:55:00'),
                           'x1': Timestamp('2024-07-11 14:50:00'),
                           'y0': 52074.8,
                           'y1': 52244.7},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-15 13:40:00'),
                           'x1': Timestamp('2024-07-15 14:10:00'),
                           'y0': 52543.3,
                           'y1': 52662.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-23 09:35:00'),
                           'x1': Timestamp('2024-07-23 10:30:00'),
                           'y0': 52088.55,
                           'y1': 52321.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-24 10:20:00'),
                           'x1': Timestamp('2024-07-24 10:55:00'),
                           'y0': 51361.35,
                           'y1': 51720.25},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
                           'x0': Timestamp('2024-07-29 09:45:00'),
                           'x1': Timestamp('2024-07-29 10:45:00'),
                           'y0': 51751.15,
                           'y1': 51954.4},
                          {'fillcolor': 'rgba(0,0,0,0)',
                           'line': {'color': 'black'},
                           'type': 'rect',
      

In [21]:
def detect_consecutive_ibars(df, max_inside_bars=12):
    consecutive_ibars = [False] * len(df)
    i = 0
    while i < len(df) - 1:
        previous_candle = df.iloc[i]

        # Start looking for inside bars after the current candle
        j = i + 1
        count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            count += 1
            j += 1
        
        # Check if the count of inside bars is within the allowed limit
        if count > 3 and count <= max_inside_bars:
            for k in range(i+1, j):
                consecutive_ibars[k] = True
            # Mark the larger candlestick as well
            consecutive_ibars[i] = True
        # Move index to the end of the current sequence
        i = j
                
    return consecutive_ibars


df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)

# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty
    else:
        i += 1

# Show the figure
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
def detect_consecutive_ibars(df):
    consecutive_ibars = [False] * len(df)
    for i in range(len(df)-1):
        previous_candle = df.iloc[i]

        j = i + 1
        count = 0
        while j < len(df) and \
              (df.iloc[j]['high'] < previous_candle['high']) and \
              (df.iloc[j]['low'] > previous_candle['low']) and \
              (df.iloc[j]['close'] < previous_candle['high']) and \
              (df.iloc[j]['open'] > previous_candle['low']):
            
            count += 1
            j += 1
        
        if count > 3:
            for k in range(i+1, j):
                consecutive_ibars[k] = True
            # Mark the larger candlestick as well
            consecutive_ibars[i] = True
                
    return consecutive_ibars

df['ConsecutiveInsideBars'] = detect_consecutive_ibars(df)


# Plot empty squares around consecutive inside bars
i = 0
while i < len(df):
    if df['ConsecutiveInsideBars'][i]:
        start = i
        while i < len(df) and df['ConsecutiveInsideBars'][i]:
            i += 1
        end = i - 1
        
        # Determine the x0 and x1 values, considering the edge cases
        x0 = df['date'][start - 1] if start > 0 else df['date'][start]
        x1 = df['date'][end + 1] if end < len(df) - 1 else df['date'][end]

        fig.add_shape(type="rect",
                      x0=x0, y0=min(df['low'][start:end+1]),
                      x1=x1, y1=max(df['high'][start:end+1]),
                      line=dict(color="black"),
                      fillcolor="rgba(0,0,0,0)")  # Make the box empty
    else:
        i+=1

# Show the figure
fig.show()
